# Updates Summary

##  Introduced a `Priors` Class for MCMC and MultiNest

- Streamlined the test_module by separating all prior-related components into a dedicated Priors module.
- Implemented dedicated classes to improve the organization and management of prior settings.
- Enhanced versatility to support different instrument modes and cloud configurations.
- Automated the generation of priors based on the retrieval parameter dictionary.
- Simplified the process for users to define and modify prior ranges for retrieved parameters.




In [1]:
import os 
# os.chdir("/Users/fw23aao/brewsteretrieval/brewster_v2.1")

import test_module
import utils
from collections import namedtuple
import numpy as np
import retrieval_run
import settings
import cloud_dic
import Priors
import utils

## 1. test_module ---> testkit+nestkit




## 2. utils:
###  Five new class in utils:
utils.Instrument

utils.Retrieval_params

utils.ModelConfig

utils.IOConfig

utils.ArgsGen

## 3. retrieval_run:
satrt mcmc or multinest smapling

# 1. mcmc template

In [2]:
# instrument

fwhm=555
wavelength_range=[0.95,2.85]
ndata=1
Rfile='./examples/example_data/G570D_R_file.txt'



#retrieval_params

##gas
chemeq=0
# gaslist = ['h2o','co','tio','vo','crh','feh','na','k']
gaslist = ['h2o','ch4']
gastype_list=['U','U']
ptype=7  


cloudpath='/Users/fw23aao/Downloads/cloud_pickles/'

## clouds

do_clouds=1
npatches=1
cloud_name=['MgSiO3.mieff'] #Mg2SiO4 MgSiO3
# cloud_name=['grey'] 
cloud_type=["slab"]
particle_dis=["hansen"] #"hansen" "log_normal"  "log_normal"

cloudpatch_index=[[1]]


# ModelConfig:
do_fudge=1
samplemode='mcmc'

instrument_instance = utils.Instrument(fwhm,wavelength_range,ndata,None, Rfile)
re_params = utils.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloud_name,cloud_type,cloudpatch_index,particle_dis, instrument_instance)
model_config_instance = utils.ModelConfig(samplemode,do_fudge,cloudpath=cloudpath)
io_config_instance = utils.IOConfig()




model_config_instance.dist= 5.84
model_config_instance.xlist ='data/gaslistRox.dat'
model_config_instance.xpath ='../../Linelists/'
model_config_instance.do_bff=0
model_config_instance.malk=1
model_config_instance.ch4=0
model_config_instance.update_dictionary()

In [3]:
obspec = np.asfortranarray(np.loadtxt("./examples/example_data/G570D_2MHcalib.dat",dtype='d',unpack='true')) # G570D_2MassJcalib.dat
args_instance = utils.ArgsGen(re_params,model_config_instance,instrument_instance,obspec)
settings.init(args_instance)

In [4]:
print(re_params)

retrieval_param: 
------------
- gas : ['h2o', 'ch4']
- refinement_params : ['logg', 'r2d2', 'dlambda', 'tolerance_parameter_1']
- pt : ['Tint', 'alpha', 'lndelta', 'T1', 'T2', 'T3']
- cloud : ['patch 1']
  -- patch 1: ['tau_mcs_MgSiO3', 'logp_mcs_MgSiO3', 'dp_mcs_MgSiO3', 'hansen_a_mcs_MgSiO3', 'hansen_b_mcs_MgSiO3']
- added_params: []



In [5]:
args_instance=settings.runargs
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary)

if re_params.samplemode=='mcmc':
    
    model_config_instance.ndim=len(all_params)
    model_config_instance.nwalkers=len(all_params)*16

    r2d2 = (71492e3)**2. / (model_config_instance.dist * 3.086e+16)**2.
    re_params.dictionary['refinement_params']['params']['r2d2']['distribution']=['normal',r2d2,0.1*r2d2]


    def tolerance_parameter_customized_distribution(x):
        return np.log10((np.random.rand(x)* (max(args_instance.obspec[2,:]**2)*(0.1 - 0.01))) + (0.01*min(args_instance.obspec[2,10::3]**2))) 

    for i in range(len(all_params)):
        if all_params[i].startswith('tolerance_parameter'):
            re_params.dictionary['refinement_params']['params'][all_params[i]]['distribution']=['customized',tolerance_parameter_customized_distribution]

    if model_config_instance.fresh == 0:
        p0=utils.MC_P0_gen(re_params.dictionary,model_config_instance,args_instance)

In [6]:
params_master = namedtuple('params',all_params)
params_instance = params_master(*p0[0,:])
params_instance

params(h2o=-3.92543786562619, ch4=-3.344796353270421, logg=4.633045210960393, r2d2=1.8046109891487813e-19, dlambda=-8.733765098750967e-06, tolerance_parameter_1=-32.26681405557298, Tint=1450.2677045595346, alpha=1.9235671659894775, lndelta=0.28796953422366833, T1=1156.748544201701, T2=976.5767034719229, T3=1545.3399473377183, tau_mcs_MgSiO3=10.77715148076989, logp_mcs_MgSiO3=0.3112234137885745, dp_mcs_MgSiO3=0.06195387232386811, hansen_a_mcs_MgSiO3=-1.2240660890432888, hansen_b_mcs_MgSiO3=0.24678098136061022)

In [13]:
re_params.dictionary['gas']['ch4']['params']['log_abund']['range']=[-10,0]

In [14]:
re_params.dictionary

{'gas': {'h2o': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': [-12, 0],
     'prior': None}}},
  'ch4': {'gastype': 'U',
   'params': {'log_abund': {'initialization': None,
     'distribution': ['normal', -4.0, 0.5],
     'range': [-10, 0],
     'prior': None}}}},
 'refinement_params': {'params': {'logg': {'initialization': None,
    'distribution': ['normal', 4.5, 0.1],
    'range': [0, 6],
    'prior': None},
   'r2d2': {'initialization': None,
    'distribution': ['normal', 1.5736109019500464e-19, 1.5736109019500464e-20],
    'range': [0, 1],
    'prior': None},
   'dlambda': {'initialization': None,
    'distribution': ['normal', 0, 0.001],
    'range': [-0.01, 0.01],
    'prior': None},
   'tolerance_parameter_1': {'initialization': None,
    'distribution': ['customized',
     <function __main__.tolerance_parameter_customized_distribution(x)>],
    'range': None,
    'prior': None}}},
 'pt': {'ptype'

In [15]:
Priors_instance =Priors.Priors(params_instance,re_params,instrument_instance)

In [16]:
Priors_instance.statement, Priors_instance.priors

(True, 0.0)

In [17]:
print(Priors_instance)

All priors considered in the retrieval:
------------
- Parameter Priors: Defined by user ranges
  * h2o: [-12, 0]
  * ch4: [-10, 0]
  * logg: [0, 6]
  * r2d2: [0, 1]
  * dlambda: [-0.01, 0.01]
  * tolerance_parameter_1: No prior range defined
  * Tint: [0, 5000]
  * alpha: [1, 2]
  * lndelta: [-20, 0]
  * T1: [0, 5000]
  * T2: [0, 5000]
  * T3: [0, 5000]
  * tau_mcs_MgSiO3: [0, 100]
  * logp_mcs_MgSiO3: [-4, 2.4]
  * dp_mcs_MgSiO3: No prior range defined
  * hansen_a_mcs_MgSiO3: [-3, 3]
  * hansen_b_mcs_MgSiO3: [0, 1]
--------------------
- Post-processing Priors:
  * T-profile check: (min(T) > 1.0) and (max(T) < 6000.)
  * Gas profile check: (np.sum(10.**(invmr)) < 1.0) and valid gas profiles
  * Mass and Radius check: (1.0 < M < 80 and 0.5 < Rj < 2.0)
  * Tolerance parameters: ((0.01*np.min(obspec[2,:]**2)) < 10.**tolerance_parameter < (100.*np.max(obspec[2,:]**2)))



# --------------------------------------------------------
# Todo_lsit

## 1. if the range of slab cloud thickness (dp_mcs) is no set, correlate it with logp_mcs with the relation of logp_gcs [0, (phi[logp_gcs_index] - np.log10(press[0]))]

## 2. flexible post-processing priors range 



# post_processing priors:
    
1. T_profile check (min(T) > 1.0) and (max(T) < 6000.) 

2. gas_profile check

3. Mass and R prior   

4. Cloud properties check  alrradey done automatically 

5. tolerance_parameter_1   ((0.01*np.min(obspec[2,:]**2)) < 10.**logf < (100.*np.max(obspec[2,:]**2)))

